In [13]:
import json
import nltk
import string
from langdetect import detect
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/phucpx/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
with open('data/train/evjvqa_train_lang.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
annotations = data['annotations']

In [3]:
len(annotations)

23785

In [4]:
lang_count = {
    "vi": 0,
    "en": 0,
    "ja": 0
}

for anno in tqdm(annotations):
    question = anno['question']
#     lang = detect(question)
#     anno['language'] = lang
    lang_count[anno['language']] += 1
        
lang_count

  0%|          | 0/23785 [00:00<?, ?it/s]

{'vi': 8332, 'en': 7194, 'ja': 8259}

In [15]:
def remove_punc(text):
    text = text.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def remove_stopword(text):
    stop_words = set(stopwords.words('english'))
  
    word_tokens = word_tokenize(text)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    return ' '.join(filtered_sentence)

In [16]:
for anno in tqdm(annotations):
    answer = remove_punc(anno['answer'])
    question = remove_punc(anno['question'])
    
    answer = remove_stopword(answer)
    ans_words = answer.split()
    ques_words = question.split()
    
    new_ans_words = []
    for w in ans_words:
        if w not in ques_words:
            new_ans_words.append(w)
    
    anno['short_answer'] = ' '.join(new_ans_words)    

  0%|          | 0/23785 [00:00<?, ?it/s]

In [17]:
with open('data/train/evjvqa_train_lang-short-anw.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

In [14]:
remove_stopword('in a grocery store')

['grocery', 'store']

In [24]:
with open('data/train/evjvqa_train_lang-short-anw.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
    
images = train_data['images']
annotations = train_data['annotations']

In [28]:
all_questions = []
all_annotations = []

for anno in tqdm(annotations):
    question_id = anno['id']
    image_id = anno['image_id']
    question = anno['question']
    
    all_questions.append(
        {
            'question_id': question_id,
            'image_id': image_id,
            'question': question
        }
    )
    
    question_type = None
    answer_type = "other"
    answers = [{"answer_id": question_id, "answer": anno['short_answer'], "answer_confidence": "yes"}]
    multi_choice_answer = anno['short_answer']
    
    all_annotations.append(
        {
            'question_id': question_id,
            'image_id': image_id,
            'question_type': question_type,
            'answer_type': answer_type,
            'answers': answers,
            'multi_choice_answer': multi_choice_answer
        }
    )
    
question_data = {
    'questions': all_questions
}
annotation_data = {
    'annotations': all_annotations
}

  0%|          | 0/23785 [00:00<?, ?it/s]

In [29]:
with open('./data/EVJVQA/train_questions.json', 'w', encoding='utf-8') as f:
    json.dump(question_data, f, indent=4, ensure_ascii=False)
    
with open('./data/EVJVQA/train_annotations.json', 'w', encoding='utf-8') as f:
    json.dump(annotation_data, f, indent=4, ensure_ascii=False)

In [30]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

In [36]:
# prepare image + question
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open('data/train/train-images/00000002301.jpg')
text = "what fruit is put on the table next to the girl in red t-shirt?"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# prepare inputs
encoding = processor(image, text, return_tensors="pt")

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()



print("Predicted answer:", model.config.id2label[idx])

Predicted answer: pineapple
